<a href="https://colab.research.google.com/github/akkuhanu/DeepLearning/blob/main/DeepLpractice/Gradient_Descent_dl12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GRADIENT DESCENT

In [34]:
# Important for interviews. (Core of ML Domain)
# suppose  y = m1*a + m2*b + m3*c +...+ d.
# in order to where m1 ,m2,m3.... are variables we give input.
# BUT a,b,c,d... are constants to find their values we use gradient descent.



In [35]:
# steps 

# w1 = w1 - something
# w1 = w1 - learningrate*d/dw1
# d/dw1--> derivative of loss w.r.t to w1.

# look for convex loss function --> https://am207.github.io/2017/wiki/gradientdescent.html
# we find the values that correspond to min. loss.

In [36]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [41]:
df = pd.read_csv("https://raw.githubusercontent.com/akkuhanu/DeepLearning/main/DeepLpractice/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [42]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2,random_state=25)

In [43]:
len(x_train)

22

In [44]:
df.shape

(28, 3)

In [45]:
x_train_scaled = x_train.copy()
x_train_scaled['age'] = x_train_scaled['age']/100

x_test_scaled = x_test.copy()
x_test_scaled['age'] = x_test_scaled['age']/100


In [46]:
x_train_scaled

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1
19,0.18,1
25,0.54,1
16,0.25,0
20,0.21,1
3,0.52,0


In [47]:
# scaling helps to bring age and affordability on the same scale.


In [48]:
model = keras.Sequential(
    [
        keras.layers.Dense(1,input_shape=(2,),activation='sigmoid',kernel_initializer='ones',bias_initializer='zeros')
    ]
)
# note it is zeros not zeroes
# binary crossentropy is same as log loss.

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy'])

model.fit(x_train_scaled,y_train,epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 8ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2502/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2503/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5321 - accuracy: 0.7273
Epoch 2504/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2505/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2506/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5320 - accuracy: 0.7273
Epoch 2507/5000
1/1 [==============================] - 0s 9ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2508/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.5319 - accuracy: 0.7273
Epoch 2509/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5318 - accuracy: 0.7273
Epoch 2510/5000
1/1 [==============================] - 0

In [49]:
model.evaluate(x_test_scaled,y_test)

1/1 [==============================] - 0s 183ms/step - loss: 0.3550 - accuracy: 1.0000


[0.35497748851776123, 1.0]

In [50]:
model.predict(x_test_scaled)
# if greater than 0.5 means buy the insurance.

array([[0.7054848 ],
       [0.35569546],
       [0.16827846],
       [0.47801173],
       [0.7260697 ],
       [0.8294984 ]], dtype=float32)

In [51]:
x_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [52]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [53]:
coef , intercept = model.get_weights()
print("Coeffecients",coef,"\n Bias Intercept:",intercept)

Coeffecients [[5.0608673]
 [1.4086503]] 
 Bias Intercept: [-2.913703]


In [54]:
import math
# takes one value
def sigmoid(x):
    return 1/(1+math.exp(-x))
sigmoid(10)

0.9999546021312976

In [55]:
# below code is like a derivation or you can say that we made the model with tensorflow help.
def prediction_function(age,affordability):
    age = age/100
    weighted_sum = coef[0]*age + coef[1]*affordability + intercept
    return sigmoid(weighted_sum)
prediction_function(47,1)

0.705484819775958

In [56]:
def log_loss(y_true,y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))
def sigmoid_numpy(x):
    return 1/(1+np.exp(-x))
sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [57]:
# passing a loss threshold meanibg stop when it matches.
def gradient_descent(age,affordibility,y_true,epochs,loss_thresold):
    # w1,w2,bias
    # here age is numpy array and afforability also and numpy array
    age = np.array(age)
    age = age/100 # scaling of age
    affordibility = np.array(affordibility)
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    
    for i in range (epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true,y_predicted)
        # d/dw1 ---> 1/n( summasion( xi* (y^i - yi) ) ) where xi is age or like factors.
#         in numpy for multiply us np.dot
        deriv_w1 = np.mean( np.dot( np.transpose(age) ,(y_predicted-y_true) ) )
        deriv_w2 = np.mean( np.dot( np.transpose(affordibility) ,(y_predicted-y_true) ) )
        deriv_bias = np.mean(y_predicted-y_true)
        
        w1 = w1 - rate*deriv_w1
        w2 = w2 - rate*deriv_w2
        bias = bias - rate*deriv_bias
        
        print(f'Epoch:{i},w1:{w1},w2{w2},bias:{bias},loss:{loss}')
        if(loss<=loss_thresold):
            break
    return w1,w2,bias

In [58]:
gradient_descent(x_train['age'],x_train['affordibility'],y_train,5000,0.4631)

Epoch:0,w1:0.4479679363438934,w2-0.13634124132036352,bias:-0.11341867736368583,loss:0.7113403233723417
Epoch:1,w1:0.9812558612136683,w20.9668377724411847,bias:-0.1110018417733685,loss:0.6847035508118
Epoch:2,w1:0.5432860438890404,w20.004384593070066445,bias:-0.21131519392809667,loss:0.6835888783135032
Epoch:3,w1:1.031660503214578,w20.9617958944318464,bias:-0.2126594225528631,loss:0.6672013121666804
Epoch:4,w1:0.6643520021310426,w20.10437614882580182,bias:-0.3044688785930035,loss:0.6654825340293027
Epoch:5,w1:1.1198293421267327,w20.9599528357847978,bias:-0.30822251315843996,loss:0.6532925028015711
Epoch:6,w1:0.8028502246932971,w20.179991548313022,bias:-0.3936533314399495,loss:0.6511488955709278
Epoch:7,w1:1.2304013077788996,w20.9567360173400518,bias:-0.3992455620331281,loss:0.6412898575315285
Epoch:8,w1:0.9521406008294833,w20.24017212730243642,bias:-0.47950536287644135,loss:0.6388394443962251
Epoch:9,w1:1.3541870964283542,w20.9515313020576354,bias:-0.48666107375165474,loss:0.63049538598

(5.585496109953384, 1.0520504840195588, -2.7686660363765507)

In [59]:
coef , intercept


(array([[5.0608673],
        [1.4086503]], dtype=float32), array([-2.913703], dtype=float32))

In [60]:
# customModel = myNN()
# customModel.fit(x_train_scaled,y_train,epochs=8000,loss_thresold=0.4631)
# customModel.predict(x_test_scaled)

In [74]:
class myNN:
    def __init__(self):
        self.w1 = 1
        self.w2 = 1
        self.bias = 0
    
    def fit(self,x,y,epochs,loss_thresold):
        self.w1,self.w2,self.bias = self.gradient_descent(x['age'],x['affordibility'],y,epochs,loss_thresold)
        
    # passing a loss threshold meanibg stop when it matches.
    def gradient_descent(self,age,affordibility,y_true,epochs,loss_thresold):
        # w1,w2,bias
        # here age is numpy array and afforability also and numpy array
        age = np.array(age)
        age = age/100 # scaling of age
        affordibility = np.array(affordibility)
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range (epochs):
            weighted_sum = w1*age + w2*affordibility + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true,y_predicted)
            # d/dw1 ---> 1/n( summasion( xi* (y^i - yi) ) ) where xi is age or like factors.
            #in numpy for multiply us np.dot
            deriv_w1 = np.mean( np.dot( np.transpose(age) ,(y_predicted-y_true) ) )
            deriv_w2 = np.mean( np.dot( np.transpose(affordibility) ,(y_predicted-y_true) ) )
            deriv_bias = np.mean(y_predicted-y_true)
            w1 = w1 - rate*deriv_w1
            w2 = w2 - rate*deriv_w2
            bias = bias - rate*deriv_bias
            print(f'Epoch:{i},w1:{w1},w2{w2},bias:{bias},loss:{loss}')
            if(loss<=loss_thresold):
                break
        return w1,w2,bias

In [75]:
customModel = myNN()
# x_train_scaled['age']
customModel.fit( x_train_scaled , y_train , epochs=8000 , loss_thresold=0.4631)


Streaming output truncated to the last 5000 lines.
Epoch:3000,w1:15.015568032412176,w21.6769861308489837,bias:-1.1530514521481172,loss:0.6128211396035492
Epoch:3001,w1:15.020230755794607,w21.6769834258278,bias:-1.153068553828188,loss:0.6128191625487938
Epoch:3002,w1:15.02489345468021,w21.6769807208592356,bias:-1.153085655443508,loss:0.6128171855148102
Epoch:3003,w1:15.029556129069086,w21.6769780159432903,bias:-1.1531027569940775,loss:0.6128152085015985
Epoch:3004,w1:15.034218778961336,w21.6769753110799623,bias:-1.1531198584798963,loss:0.6128132315091587
Epoch:3005,w1:15.038881404357062,w21.6769726062692518,bias:-1.1531369599009642,loss:0.6128112545374904
Epoch:3006,w1:15.043544005256367,w21.6769699015111574,bias:-1.1531540612572815,loss:0.6128092775865935
Epoch:3007,w1:15.04820658165935,w21.6769671968056787,bias:-1.1531711625488479,loss:0.612807300656468
Epoch:3008,w1:15.052869133566116,w21.6769644921528148,bias:-1.1531882637756634,loss:0.6128053237471135
Epoch:3009,w1:15.0575316609767